In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as	plt
import seaborn as sns
import json
from collections import defaultdict

In [251]:
df = pd.read_csv('train', sep=' ', on_bad_lines='skip', engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])
dev= pd.read_csv('dev.in', sep=' ', on_bad_lines='skip', engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])
labels_list = df['Label'].tolist()
dev_list= dev['Token'].tolist()
labels_list.insert(0,"START")
labels_list.append("STOP")
token_label_list = df[['Token', 'Label']].values.tolist()


In [40]:
#display(df)
print(dev_list)
# print(labels_list)
# print(token_label_list)

['Plato', 'degustación', ':', 'un', 'poco', 'abundante', 'de', 'más', ',', 'pero', 'bien', 'cocinado', '.', 'restaurante', 'excelente', 'con', 'carne', 'de', 'alta', 'calidad', '.', 'Las', 'posibilidades', 'en', 'el', 'restaurante', 'son', 'fundamentalmente', 'tres', ';', 'carta', 'normal', ',', 'menú', 'degustacion', 'y', 'una', 'opción', 'intermedia', 'que', 'es', 'una', 'selección', 'de', 'primeros', 'y', 'postres', 'y', 'carta', 'para', 'el', 'segundo', '.', 'No', 'perderse', 'el', 'sorbete', 'de', 'mojito', '.', 'para', 'mi', 'perfecto', '!', 'Devolucion', 'a', 'cocina', ',', 'amabilidad', 'de', 'camarera', ',', 'requerimiento', 'de', 'cuenta', 'y', 'adios', '.', 'Así', 'como', 'el', 'romesco', ',', 'que', 'era', 'un', 'poco', '"', 'de', 'bote', '"', '.', 'Destacar', 'los', 'arroces', ',', 'la', 'caldereta', 'de', 'bogavante', ',', 'las', 'zamburiñas', 'al', 'horno', 'y', 'los', 'platos', 'de', '"', 'picoteo', '"', 'y', 'los', 'pescados', 'en', 'general', '.', 'Somos', 'clientes',

In [5]:
df_token = df.loc[:,["Token"]]
df_label = df.loc[:,["Label"]]

arr_token = (df_token.squeeze()).to_numpy()
arr_label = (df_label.squeeze()).to_numpy()

token_all = df_token.squeeze().unique()
label_all = df_label.squeeze().unique()

In [6]:
y_all = pd.DataFrame(df_label.value_counts()).reset_index()
y_all.columns = ['Label', 'Count']
y_all["Location"] = [np.where(arr_label == i)[0] for i in label_all]

print(len(y_all))
print(type(y_all))
display(y_all)

7
<class 'pandas.core.frame.DataFrame'>


,Label,Count,Location
0,O,29035,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,B-positive,1160,"[16, 19, 22, 25, 60, 63, 75, 86, 90, 113, 204,..."
2,B-negative,381,"[39, 98, 124, 131, 133, 137, 323, 382, 472, 62..."
3,I-positive,314,"[153, 591, 685, 1053, 1180, 1467, 1973, 1994, ..."
4,I-negative,171,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ..."
5,B-neutral,72,"[313, 314, 315, 937, 1400, 1401, 1939, 2054, 2..."
6,I-neutral,43,"[802, 1171, 1626, 1627, 1724, 1725, 1726, 1727..."


In [7]:
x_all = pd.DataFrame(token_all, columns=['Token'])

# Need to check more, this takes >1m to do
new_x_all = pd.DataFrame({'Token': np.repeat(x_all['Token'], 7)}).reset_index()
new_x_all.columns = ['Index', 'Token']
new_x_all["Label"] = np.tile(label_all, int(len(new_x_all)/7))
count_x_all = []
location_x_all = []

for i in token_all:
    for j in label_all:
        cond_1 = arr_token == i
        cond_2 = arr_label == j
        loc = (np.where(cond_1 & cond_2)[0]).tolist()
        # df_ij = df.iloc[loc, :]
        count_x_all += [len(loc)]
        location_x_all += [loc]
new_x_all["Count"] = count_x_all
new_x_all["Location"] = location_x_all

print(len(new_x_all))
print(type(new_x_all))
display(new_x_all)

35280
<class 'pandas.core.frame.DataFrame'>


,Index,Token,Label,Count,Location
0,0,Estuvimos,O,6,"[0, 13026, 13936, 17095, 27870, 29879]"
1,0,Estuvimos,B-positive,0,[]
2,0,Estuvimos,B-negative,0,[]
3,0,Estuvimos,B-neutral,0,[]
4,0,Estuvimos,I-neutral,0,[]
...,...,...,...,...,...
35275,5039,inolvidable,B-negative,0,[]
35276,5039,inolvidable,B-neutral,0,[]
35277,5039,inolvidable,I-neutral,0,[]
35278,5039,inolvidable,I-positive,0,[]


$1. \:\:\: e(x \mid y)=\frac{\operatorname{Count}(y \rightarrow x)}{\operatorname{Count}(y)}$

In [156]:
def emm_par(x, y, emission_params, label_counts, k=1):
    if (x, y) in emission_params:
        return emission_params[(x, y)]
    else:
        return k / (label_counts[y] + k * len(label_counts))

In [231]:
# Calculate label counts
label_counts = y_all.set_index('Label')['Count'].to_dict()

# Ensure all possible labels are included in the label_counts dictionary
for label in label_all:
    if label not in label_counts:
        label_counts[label] = 0

def calculate_emission_params(token_label_list, new_x_all, y_all, k=1):
    emission_params = {}
    
    # Calculate label counts
    label_counts = y_all.set_index('Label')['Count'].to_dict()
    for label in label_all:
        if label not in label_counts:
            label_counts[label] = 0
    # Calculate emission parameters with Laplace smoothing
    for x, y in token_label_list:
        if (x, y) not in emission_params:
            cond_1 = new_x_all["Token"] == x
            cond_2 = new_x_all["Label"] == y
            num = new_x_all.loc[cond_1 & cond_2, "Count"].values[0]
            emission_params[(x,y)] = (num + k) / (label_counts[y] + k * len(label_counts))

    return emission_params

In [232]:
emission_params_dict = calculate_emission_params(token_label_list, new_x_all, y_all, k=1)

print(emission_params_dict)

{('Estuvimos', 'O'): 0.0002410302320776806, ('hace', 'O'): 0.0009296880380139109, ('poco', 'O'): 0.0019282418566214448, ('mi', 'O'): 0.0025136009916672405, ('pareja', 'O'): 0.0004820604641553612, ('y', 'O'): 0.035293712554231804, ('yo', 'O'): 0.001274016940982026, ('comiendo', 'O'): 0.00037876179326492665, ('resultó', 'O'): 0.00017216445148405756, ('todo', 'O'): 0.0039942152744301355, ('muy', 'O'): 0.013669857447834171, ('bien', 'O'): 0.0057158597892707115, (',', 'O'): 0.05733076234419117, ('tanto', 'O'): 0.0013773156118724605, ('la', 'O'): 0.026031265064389505, ('comida', 'B-positive'): 0.1456726649528706, ('el', 'O'): 0.022140348460849803, ('vino', 'B-positive'): 0.005998286203941731, ('trato', 'B-positive'): 0.038560411311053984, ('decoración', 'B-positive'): 0.006855184233076264, ('…', 'O'): 0.0015839129536533297, ('nos', 'O'): 0.005061634873631293, ('gustó', 'O'): 0.00041319468356173817, ('mucho', 'O'): 0.0018593760760278219, ('.', 'O'): 0.0559190138420219, ('Por', 'O'): 0.0010674

In [177]:
a = emm_par("Estuvimos", "O")
print(a)

b = emm_par("estuvimos", "O")
print(b)

TypeError: emm_par() missing 1 required positional argument: 'emission_params'

$2. \:\:\:e(x \mid y)=\left\{\begin{array}{cc}
\frac{\operatorname{count}(y \rightarrow x)}{\operatorname{Count}(y)+k} & \text { If the word token } x \text { appears in the training set } \\
\frac{k}{\operatorname{Count}(y)+k} & \text { If word token } x \text { is the special token \#UNK\# }
\end{array}\right.$

In [271]:
def emm_par_test(x, y, new_x_all, y_all, k=1, unk_token='#UNK#'):
    # x is a token
    # y is a label
    
    if x == unk_token:
        num = k
    elif x in new_x_all["Token"].unique():
        cond_1 = new_x_all["Token"] == x
        cond_2 = new_x_all["Label"] == y
        num = new_x_all.loc[cond_1 & cond_2, "Count"].values[0]
    else:
        num = k
    
    cond_3 = y_all["Label"] == y
    denom = y_all.loc[cond_3, "Count"].values[0] + k
    
    return num / denom


In [205]:
a = emm_par_test(",", "I-negative")
print(a)

b = emm_par_test("Kontollllllllllll", "I-neutral")
print(b)

TypeError: emm_par_test() missing 2 required positional arguments: 'new_x_all' and 'y_all'

$3. \:\:\:y^*=\underset{y}{\arg \max } \:e(x \mid y)$

In [206]:
df_y = pd.DataFrame()

emm_par_score_all = []
y_star_all = []
token_all = np.append(token_all, "#UNK#")
print(token_all)

['Estuvimos' 'hace' 'poco' ... '#UNK#' '#UNK#' '#UNK#']


In [208]:
y_star_all = []
emm_par_score_all = []

for i in token_all:
    emm_par_score = 0
    y_star = ""
    for j in label_all:
        emm_par_score_temp = emm_par_test(i, j, new_x_all, y_all)
        if emm_par_score_temp > emm_par_score:
            emm_par_score = emm_par_score_temp
            y_star = j
    y_star_all.append(y_star)
    emm_par_score_all.append(emm_par_score)


In [210]:
print(emm_par_score_all)
print(y_star_all)

[0.00020664003306240529, 0.0008954401432704229, 0.0018942003030720485, 0.0024796803967488635, 0.00044772007163521146, 0.0352665656426505, 0.005813953488372093, 0.00034440005510400884, 0.00013776002204160352, 0.005813953488372093, 0.013638242182118749, 0.005682600909216145, 0.05730816916930707, 0.0013431602149056344, 0.0380952380952381, 0.14556416881998277, 0.022727272727272728, 0.015706806282722512, 0.03789836347975883, 0.006029285099052541, 0.002617801047120419, 0.005028240804518529, 0.0003788400606144097, 0.005813953488372093, 0.055896128943380634, 0.0010332001653120264, 0.00017220002755200442, 0.00020664003306240529, 0.022727272727272728, 0.00013776002204160352, 0.005813953488372093, 0.012708362033337925, 0.0017908802865408459, 0.009298801487808239, 0.011627906976744186, 6.888001102080176e-05, 0.029101804656288744, 3.444000551040088e-05, 0.00017220002755200442, 0.00020664003306240529, 0.00017220002755200442, 6.888001102080176e-05, 0.023255813953488372, 6.888001102080176e-05, 0.00451

In [214]:
df_y["Token"] = token_all
df_y["y*"] = y_star_all
df_y["Max e(x|y)"] = emm_par_score_all

display(df_y)

for i in label_all:
    print(emm_par_test("sin", i, new_x_all, y_all))


,Token,y*,Max e(x|y)
0,Estuvimos,O,0.000207
1,hace,O,0.000895
2,poco,O,0.001894
3,mi,O,0.002480
4,pareja,O,0.000448
...,...,...,...
5040,#UNK#,I-neutral,0.022727
5041,#UNK#,I-neutral,0.022727
5042,#UNK#,I-neutral,0.022727
5043,#UNK#,I-neutral,0.022727


0.0028240804518528725
0.0
0.0
0.0
0.0
0.0
0.005813953488372093


In [215]:
df_y_dict = {row['Token']: (row['y*'], row['Max e(x|y)']) for row in df_y.to_dict(orient='records')}
print(df_y_dict)
print(df_y_dict[","])

{'Estuvimos': ('O', 0.00020664003306240529), 'hace': ('O', 0.0008954401432704229), 'poco': ('O', 0.0018942003030720485), 'mi': ('O', 0.0024796803967488635), 'pareja': ('O', 0.00044772007163521146), 'y': ('O', 0.0352665656426505), 'yo': ('I-negative', 0.005813953488372093), 'comiendo': ('O', 0.00034440005510400884), 'resultó': ('O', 0.00013776002204160352), 'todo': ('I-negative', 0.005813953488372093), 'muy': ('O', 0.013638242182118749), 'bien': ('O', 0.005682600909216145), ',': ('O', 0.05730816916930707), 'tanto': ('O', 0.0013431602149056344), 'la': ('I-positive', 0.0380952380952381), 'comida': ('B-positive', 0.14556416881998277), 'el': ('I-neutral', 0.022727272727272728), 'vino': ('B-negative', 0.015706806282722512), 'trato': ('B-positive', 0.03789836347975883), 'decoración': ('B-positive', 0.006029285099052541), '…': ('B-negative', 0.002617801047120419), 'nos': ('O', 0.005028240804518529), 'gustó': ('O', 0.0003788400606144097), 'mucho': ('I-negative', 0.005813953488372093), '.': ('O'

Testing

In [216]:
df_test = pd.read_csv('dev.out', sep=' ', on_bad_lines='skip', engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

In [217]:
display(df_test)

,Token,Label
0,Plato,B-positive
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
...,...,...
4307,sin,O
4308,dejarte,O
4309,la,O
4310,cartera,O


In [218]:
df_test_token = df_test.loc[:,["Token"]]
df_test_label = df_test.loc[:,["Label"]]

arr_test_token = (df_test_token.squeeze()).to_numpy()
arr_test_label = (df_test_label.squeeze()).to_numpy()

token_test_all = df_test_token.squeeze().unique()
label_test_all = df_test_label.squeeze().unique()

In [219]:
test_pred = []
new_token = []
for i in arr_test_token:
    try:
        pred = df_y_dict[i][0]
        test_pred += [pred]
    except:
        pred = df_y_dict["#UNK#"][0]
        test_pred += [pred]
        new_token += [i]

In [220]:
df_test["Predicted Label"] = test_pred
display(df_test)

arr_test_pred_label = ((df_test.loc[:,["Predicted Label"]]).squeeze()).to_numpy()

,Token,Label,Predicted Label
0,Plato,B-positive,B-negative
1,degustación,I-positive,I-positive
2,:,O,O
3,un,O,O
4,poco,O,O
...,...,...,...
4307,sin,O,I-negative
4308,dejarte,O,I-neutral
4309,la,O,I-positive
4310,cartera,O,I-neutral


In [221]:
entity_loc = (np.where(arr_test_label != "O")[0]).tolist()
pred_entity_loc = (np.where(arr_test_pred_label != "O")[0]).tolist()

In [222]:
print(entity_loc)
print(pred_entity_loc)

[0, 1, 13, 16, 56, 57, 58, 70, 81, 94, 97, 98, 99, 102, 103, 104, 107, 108, 109, 110, 114, 130, 131, 142, 159, 162, 170, 174, 193, 208, 209, 210, 224, 249, 258, 269, 276, 287, 320, 376, 413, 418, 419, 420, 421, 425, 431, 433, 436, 486, 514, 553, 561, 585, 587, 593, 596, 598, 640, 647, 650, 653, 689, 720, 740, 754, 755, 779, 782, 786, 826, 829, 830, 831, 838, 844, 877, 886, 889, 895, 899, 919, 923, 939, 940, 941, 942, 1019, 1029, 1034, 1036, 1140, 1146, 1157, 1158, 1159, 1160, 1161, 1167, 1177, 1185, 1266, 1293, 1298, 1306, 1310, 1454, 1463, 1483, 1508, 1516, 1517, 1520, 1523, 1526, 1574, 1590, 1601, 1616, 1637, 1638, 1639, 1642, 1645, 1664, 1665, 1701, 1704, 1776, 1777, 1801, 1810, 1815, 1829, 1842, 1843, 1844, 1860, 1866, 1868, 1885, 1890, 1897, 1901, 1934, 1940, 1949, 1956, 1979, 2039, 2044, 2049, 2074, 2169, 2183, 2217, 2218, 2219, 2225, 2236, 2241, 2246, 2257, 2272, 2280, 2294, 2295, 2296, 2299, 2318, 2418, 2421, 2427, 2433, 2434, 2435, 2462, 2463, 2464, 2506, 2508, 2533, 2537, 254

In [223]:
total_num_pred_e = len(pred_entity_loc)
total_num_gold_e = len(entity_loc)

df_test_entity = ((df_test.loc[entity_loc,:]).copy()).reset_index(drop=True)
display(df_test_entity)
df_test_entity_label = df_test_entity.loc[:,["Label"]]
df_test_entity_pred_label = df_test_entity.loc[:,["Predicted Label"]]

c_pred_entity_loc = (np.where((df_test_entity_label.squeeze()).to_numpy() == (df_test_entity_pred_label.squeeze()).to_numpy())[0]).tolist()
print(c_pred_entity_loc)

total_num_c_pred_e = len(c_pred_entity_loc)

,Token,Label,Predicted Label
0,Plato,B-positive,B-negative
1,degustación,I-positive,I-positive
2,restaurante,B-positive,B-negative
3,carne,B-positive,B-negative
4,sorbete,B-positive,B-positive
...,...,...,...
309,servicio,B-positive,B-positive
310,servicio,B-negative,B-positive
311,mesa,B-negative,B-negative
312,camareros,B-positive,B-negative


[1, 4, 5, 7, 9, 11, 15, 17, 20, 21, 22, 23, 26, 27, 28, 34, 36, 38, 39, 41, 42, 45, 46, 48, 53, 55, 56, 58, 59, 60, 61, 62, 63, 67, 69, 75, 76, 79, 80, 89, 98, 101, 104, 110, 112, 114, 118, 122, 123, 124, 128, 129, 131, 135, 136, 138, 139, 141, 142, 143, 144, 147, 148, 150, 151, 156, 159, 160, 162, 164, 166, 168, 170, 171, 178, 180, 181, 182, 183, 185, 186, 187, 200, 201, 202, 203, 206, 208, 210, 211, 217, 219, 221, 223, 226, 230, 233, 234, 235, 236, 239, 245, 246, 250, 251, 252, 255, 256, 257, 258, 263, 264, 267, 271, 272, 273, 274, 280, 283, 290, 294, 298, 300, 301, 303, 304, 309, 311, 313]


In [224]:
print(total_num_pred_e)
print(total_num_gold_e)
print(total_num_c_pred_e)

1690
314
129


$\text { Precision }=\frac{\text { Total number of correctly predicted entities }}{\text { Total number of predicted entities }}$

In [195]:
precision = total_num_c_pred_e/total_num_pred_e

$\text { Recall }=\frac{\text { Total number of correctly predicted entities }}{\text { Total number of gold entities }}$

In [196]:
recall = total_num_c_pred_e/total_num_gold_e

$F=\frac{2}{1 / \text { Precision }+1 / \text { Recall }}$

In [197]:
f_score = 2/((1/precision)+(1/recall))

In [198]:
print(precision)
print(recall)
print(f_score)

0.07633136094674556
0.410828025477707
0.1287425149700599


In [256]:
def estimate_transition_parameters(labels):
    """
    Estimates the transition parameters from a sequence of labels using Maximum Likelihood Estimation (MLE).

    Args:
        labels (list): A list of labels representing a sequence of tags.

    Returns:
        dict: A dictionary containing transition probabilities as {('previous_tag', 'current_tag'): probability}.
    """

    # Count the occurrences of each (previous_tag, current_tag) pair
    transition_counts = {}

    for i in range(len(labels) - 1):
        prev_tag = labels[i]
        current_tag = labels[i + 1]
        # Increment the count of the current_tag given the previous_tag
        transition_counts[(prev_tag, current_tag)] = transition_counts.get((prev_tag, current_tag), 0) + 1

    # Calculate transition probabilities based on MLE with smoothing
    transition_probabilities = {}
    prev_tag_counts = {'START': 0}  # Initialize count of 'START' tag

    for (_, current_tag), count in transition_counts.items():
        prev_tag = current_tag
        # Increment the count of the previous tag
        prev_tag_counts[prev_tag] = prev_tag_counts.get(prev_tag, 0) + count

    epsilon = 1e-10  # Small smoothing constant
    for (prev_tag, current_tag), count in transition_counts.items():
        # Calculate the probability q(current_tag | prev_tag) using MLE formula with smoothing
        prev_tag_count = prev_tag_counts[prev_tag]
        transition_probabilities[(prev_tag, current_tag)] = (count + epsilon) / (prev_tag_count + epsilon * len(transition_counts))

    return transition_probabilities


In [257]:
transition_params_dict=estimate_transition_parameters(labels_list)
print(transition_params_dict)

{('START', 'O'): 500000000.04999995, ('O', 'O'): 0.9445496814189154, ('O', 'B-positive'): 0.03984845875667368, ('B-positive', 'O'): 0.8801724137916722, ('O', 'B-negative'): 0.013122094024455785, ('B-negative', 'O'): 0.8215223097072362, ('O', 'B-neutral'): 0.0024453246082347204, ('B-neutral', 'I-neutral'): 0.2083333333289352, ('I-neutral', 'I-neutral'): 0.6511627906697133, ('I-neutral', 'O'): 0.34883720928842615, ('B-positive', 'I-positive'): 0.11637931034471313, ('I-positive', 'I-positive'): 0.5700636942642034, ('I-positive', 'O'): 0.4299363057300641, ('B-neutral', 'O'): 0.7916666666460649, ('B-negative', 'I-negative'): 0.1784776902880395, ('I-negative', 'O'): 0.3976608187093841, ('I-negative', 'I-negative'): 0.6023391812800896, ('B-positive', 'B-neutral'): 0.000862068965601962, ('B-positive', 'B-positive'): 0.002586206896633472, ('O', 'STOP'): 3.444119166867336e-05}


In [ ]:
import numpy as np

def viterbi(words, transition_params, emission_params):
    n = len(words)
    tags = list(transition_params.keys())  # Use transition_params keys for tags
    num_tags = len(tags)

    # Initialize the dynamic programming table and backpointers
    dp = np.full((num_tags, n), -np.inf)  # Use negative infinity for log probabilities
    backpointers = np.zeros((num_tags, n), dtype=int)

    # Initialization step
    for i, tag in enumerate(tags):
        dp[i, 0] = np.log(transition_params.get(('START', tag), 1e-10)) + np.log(emission_params.get((tag, words[0]), 1e-10))

    # Recursion step
    for t in range(1, n):
        for i, tag in enumerate(tags):
            max_prob = -np.inf
            max_prev = -1
            for j, prev_tag in enumerate(tags):
                transition_prob = np.log(transition_params.get((prev_tag, tag), 1e-10))
                emission_prob = np.log(emission_params.get((words[t], tag), 1e-10))  # Swap (words[t], tag) in emission_params
                prob = dp[j, t - 1] + transition_prob + emission_prob
                if prob > max_prob:
                    max_prob = prob
                    max_prev = j
            dp[i, t] = max_prob
            backpointers[i, t] = max_prev

    # Termination step
    max_prob = -np.inf
    end_tag = ''
    for i, tag in enumerate(tags):
        transition_prob = np.log(transition_params.get((tag, 'STOP'), 1e-10))
        prob = dp[i, n - 1] + transition_prob
        if prob > max_prob:
            max_prob = prob
            end_tag = tag

    # Backtrack to find the best tag sequence
    best_tags = [end_tag]
    prev_tag_index = tags.index(end_tag)
    for t in range(n - 1, 0, -1):
        prev_tag_index = backpointers[prev_tag_index, t]
        best_tags.insert(0, tags[prev_tag_index])

    return best_tags

In [293]:
def read_sentences(file_path):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        sentence = []
        for line in file:
            line = line.strip()
            if not line:
                sentences.append(sentence)
                sentence = []
            else:
                word = line.split()[0]
                sentence.append(word)
        if sentence:
            sentences.append(sentence)
    return sentences

dev_sentences = read_sentences('dev.in')
print(dev_sentences)


[['Plato', 'degustación', ':', 'un', 'poco', 'abundante', 'de', 'más', ',', 'pero', 'bien', 'cocinado', '.'], ['restaurante', 'excelente', 'con', 'carne', 'de', 'alta', 'calidad', '.'], ['Las', 'posibilidades', 'en', 'el', 'restaurante', 'son', 'fundamentalmente', 'tres', ';', 'carta', 'normal', ',', 'menú', 'degustacion', 'y', 'una', 'opción', 'intermedia', 'que', 'es', 'una', 'selección', 'de', 'primeros', 'y', 'postres', 'y', 'carta', 'para', 'el', 'segundo', '.'], ['No', 'perderse', 'el', 'sorbete', 'de', 'mojito', '.'], ['para', 'mi', 'perfecto', '!'], ['Devolucion', 'a', 'cocina', ',', 'amabilidad', 'de', 'camarera', ',', 'requerimiento', 'de', 'cuenta', 'y', 'adios', '.'], ['Así', 'como', 'el', 'romesco', ',', 'que', 'era', 'un', 'poco', '"', 'de', 'bote', '"', '.'], ['Destacar', 'los', 'arroces', ',', 'la', 'caldereta', 'de', 'bogavante', ',', 'las', 'zamburiñas', 'al', 'horno', 'y', 'los', 'platos', 'de', '"', 'picoteo', '"', 'y', 'los', 'pescados', 'en', 'general', '.'], ['So

In [294]:
def write_output(file_path, sentences, predicted_tags):
    with open(file_path, 'w', encoding='utf-8') as file:
        for i, sentence in enumerate(sentences):
            for j, word in enumerate(sentence):
                file.write(f'{word} {predicted_tags[i][j]}\n')
            file.write('\n')

predicted_tags_list = []

for sentence in dev_sentences:
    predicted_tags = viterbi(sentence, transition_params_dict,emission_params_dict)
    predicted_tags_list.append(predicted_tags)

write_output('dev.p23.out', dev_sentences, predicted_tags_list)

KeyboardInterrupt: 